#Only run this in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import small_rubics as rubics
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from small_rubics_neural_networks import student_network

task = rubics.rubics_task
setup = rubics.rubics_setup

student_network = student_network()
student_template = student(task, 40,student_network)
t = teacher(setup)
c = classroom(task, setup, t, student_template, n_students=1, max_steps=4)
c.run_training_batch(1)


Finished step 0, 1 out of 1 remain open.
Finished step 1, 1 out of 1 remain open.
Finished step 2, 1 out of 1 remain open.
Finished step 3, 1 out of 1 remain open.
1/1 [==============================] - 1s 600ms/step - loss: 0.9421 - eval_output_loss: 0.2823 - policy_output_loss: 0.0810


In [5]:
for i in range(20):
    c.run_training_batch(1)

  BB    
  OO    
RBWWGOYY
RBWWGOYY
  RR    
  GG    

Finished step 0, 1 out of 1 remain open.
Finished step 1, 0 out of 1 remain open.
Finished step 2, 0 out of 1 remain open.
Finished step 3, 0 out of 1 remain open.
1/1 [==============================] - 0s 7ms/step - loss: 0.1690 - eval_output_loss: 1.6078e-04 - policy_output_loss: 0.1270
Finished step 0, 1 out of 1 remain open.
Finished step 1, 0 out of 1 remain open.
Finished step 2, 0 out of 1 remain open.
Finished step 3, 0 out of 1 remain open.
1/1 [==============================] - 0s 5ms/step - loss: 0.1658 - eval_output_loss: 1.5557e-04 - policy_output_loss: 0.1239
Finished step 0, 1 out of 1 remain open.
Finished step 1, 0 out of 1 remain open.
Finished step 2, 0 out of 1 remain open.
Finished step 3, 0 out of 1 remain open.
1/1 [==============================] - 0s 5ms/step - loss: 0.1616 - eval_output_loss: 1.5106e-04 - policy_output_loss: 0.1199
Finished step 0, 1 out of 1 remain open.
Finished step 1, 0 out of 1 remain